In [7]:
from pymongo import MongoClient
from pandas.io.json import json_normalize
import pandas as pd

In [8]:
client = MongoClient ('localhost', 27017)
db = client['companies']

In [9]:
#Listado de categorias de las compañias
categorias = db.companies.distinct('category_code');
print(list(categorias))

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'mobile', 'music', 'social', 'search', 'messaging', 'advertising', 'photo_video', 'security', 'finance', 'ecommerce', 'travel', 'hardware', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


In [37]:
categorias = ['web', 'software', 'games_video', 'social', 'design', 'search']
campos = {'name': 1,
          'number_of_employees': 1,
          'category_code': 1,
          'acquisition': 1,
          'founded_year': 1,
          'funding_rounds': 1,
          'offices.latitude': 1,
          'offices.longitude': 1,
          'offices.city': 1,
          'offices.country_code': 1}

def buscar_startup(categorias, campos):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$lt':100}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},
                                       {'founded_year':{'$gte': 2010}}]},
                              campos)
    return datos

def buscar_big(categorias):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$gt': 1000}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},                                       
                                       {'founded_year':{'$gte': 1980}}]},
                              {'name': 1,
                               'number_of_employees': 1,
                               'category_code': 1,
                               'acquisition': 1,
                               'founded_year': 1,
                               'funding_rounds': 1,
                               'offices.latitude': 1,
                               'offices.longitude': 1,
                               'offices.city': 1,
                               'offices.country_code': 1})
    return datos


In [38]:
big_office_data = json_normalize(data = buscar_big(categorias), 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
big_office_data

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,Menlo Park,USA,37.416050,-122.151801,Facebook,social,2004,5299
1,Dublin,IRL,53.344104,-6.267494,Facebook,social,2004,5299
2,New York,USA,40.755716,-73.979247,Facebook,social,2004,5299
3,San Francisco,USA,37.776805,-122.416924,Twitter,social,2006,1300
4,San Jose,USA,37.295005,-121.930035,eBay,web,1995,15000
5,Sunnyvale,USA,37.418531,-122.025485,Yahoo!,web,1994,13600
6,Mountain View,USA,37.423258,-122.073635,LinkedIn,social,2003,4800
7,Mountain View,USA,37.421972,-122.084143,Google,search,1998,28000
8,Ann Arbor,USA,42.280988,-83.748882,Google,search,1998,28000
9,Atlanta,USA,33.781466,-84.387519,Google,search,1998,28000


In [39]:
startup_office_data = json_normalize(data = buscar_startup(categorias, campos), 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
startup_office_data

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20
1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8
2,Hannover,DEU,NaN,NaN,headr,web,2012,8
3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30
4,Palo Alto,USA,NaN,NaN,Simplicant,software,2012,10
5,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6
6,Van Nuys,USA,40.650291,-74.294395,CollegeConvo,web,2010,2
7,Bethesda,USA,38.989124,-77.026676,Carfeine,software,2012,5
8,Palo Alto,USA,37.444098,-122.161287,Ziippi,web,2011,6
9,Santa Monica,USA,53.544711,-113.515769,Titan Gaming,games_video,2010,18


In [40]:
data = pd.concat([startup_office_data, big_office_data])
data = data.reset_index()
data = data.dropna()
data.head(210)

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20
1,1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8
3,3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30
5,5,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6
6,6,Van Nuys,USA,40.650291,-74.294395,CollegeConvo,web,2010,2
7,7,Bethesda,USA,38.989124,-77.026676,Carfeine,software,2012,5
8,8,Palo Alto,USA,37.444098,-122.161287,Ziippi,web,2011,6
9,9,Santa Monica,USA,53.544711,-113.515769,Titan Gaming,games_video,2010,18
10,10,Vancouver,CAN,49.263588,-123.138565,Pixelmatic,games_video,2011,10
12,12,Chicago,USA,41.857204,-87.623923,JumpForward,software,2010,30


In [41]:
#data['coordenadas'] = [{'type': 'Point', 'coordinates': [x, y]} for x, y in zip(data['longitude'], data['latitude'])]

data['coordenadas'] = [[x, y] for x, y in zip(data['longitude'], data['latitude'])]
data.head(120)

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,coordenadas
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,"[-73.985506, 40.757929]"
1,1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8,"[13.4109071, 52.5013449]"
3,3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30,"[-122.323895, 37.566879]"
5,5,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6,"[-122.387799, 37.760524]"
6,6,Van Nuys,USA,40.650291,-74.294395,CollegeConvo,web,2010,2,"[-74.294395, 40.650291]"
7,7,Bethesda,USA,38.989124,-77.026676,Carfeine,software,2012,5,"[-77.026676, 38.989124]"
8,8,Palo Alto,USA,37.444098,-122.161287,Ziippi,web,2011,6,"[-122.1612868, 37.4440981]"
9,9,Santa Monica,USA,53.544711,-113.515769,Titan Gaming,games_video,2010,18,"[-113.515769, 53.544711]"
10,10,Vancouver,CAN,49.263588,-123.138565,Pixelmatic,games_video,2011,10,"[-123.138565, 49.263588]"
12,12,Chicago,USA,41.857204,-87.623923,JumpForward,software,2010,30,"[-87.623923, 41.857204]"


In [42]:
data.to_json('oficinas.json', orient="records", lines=True)

In [43]:
db_ofi = client['oficinas']

def buscar_oficinas(lng, lat, min_dist_m, max_dist_m):
    nearLocation = {
        "lng": lng,
        "lat": lat
    }
    busqueda = db_ofi.oficinas.find({
        "coordenadas": {
         "$nearSphere": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ nearLocation["lng"] , nearLocation["lat"] ]
           },
           '$minDistance': min_dist_m,
           "$maxDistance": max_dist_m, 
         }
       }
    })
    
    return busqueda

query = buscar_oficinas(-73.990869, 40.748368, 100, 5000)

resultado = pd.DataFrame(query)
resultado

,_id,category_code,city,coordenadas,country_code,founded_year,index,latitude,longitude,name,number_of_employees
0,5cd432976764028fb16e6380,web,New York,"[-73.987764, 40.744618]",USA,2010,18,40.744618,-73.987764,Yipit,23
1,5cd432976764028fb16e6387,search,New York,"[-73.985506, 40.757929]",USA,2012,0,40.757929,-73.985506,PeekYou,20
2,5cd432976764028fb16e6398,social,New York,"[-73.9792469, 40.7557162]",USA,2004,2,40.755716,-73.979247,Facebook,5299
3,5cd432976764028fb16e63aa,search,New York,"[-74.004489, 40.74222]",USA,1998,20,40.742220,-74.004489,Google,28000
4,5cd432976764028fb16e63be,web,New York,"[-73.9919311, 40.7311317]",USA,1985,75,40.731132,-73.991931,AOL,8000
5,5cd432976764028fb16e6383,software,New York,"[-73.9799007, 40.7645772]",USA,2011,20,40.764577,-73.979901,Unison Technologies,30


In [44]:
# import libraries
import folium

             
m = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)

 
# I can add marker one by one on the map
for i in range(0,len(data)):
    if  data.iloc[i]['number_of_employees'] > 999:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='blue')).add_to(m)
    else:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='green')).add_to(m)        
 

m.save('map-oficinas.html')


In [56]:
#from IPython.display import HTML
#HTML(filename='map-oficinas.html')